In [ ]:
import numpy as np
import cv2
import math
import mediapipe as mp
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import savgol_filter
import json

In [ ]:
def angle(vector_1,vector_2):
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return math.degrees(angle) #convert radians to degree

In [ ]:
keypoint_map_17 = {
    "Nose" : 0,
    "LEye" : 3,
    "REye" : 6,
    "LEar" : 9,
    "REar" : 12,
    "LShoulder" : 15,
    "RShoulder" : 18,
    "LElbow" : 21,
    "RElbow" : 24,
    "LWrist" : 27,
    "RWrist" : 30,  
    "LHip" : 33,
    "RHip" : 36,
    "LKnee" : 39,
    "RKnee" : 42,
    "LAnkle" : 45,
    "RAnkle" : 48
}

keypoint_map_18 = {
    "Nose" : 0,
    "Neck" : 3,
    "RShoulder" : 6,
    "RElbow" : 9,
    "RWrist" : 12,
    "LShoulder" : 15,
    "LElbow" : 18,
    "LWrist" : 21,
    "RHip" : 24,
    "RKnee" : 27,
    "RAnkle" : 30,
    "LHip" : 33,
    "LKnee" : 36,
    "LAnkle" : 39,
    "REye" : 42,
    "LEye" : 45,
    "REar" :48,
    "LEar" :51
}

In [ ]:
#load json file
f = open("../UPDRS_video/10_19/case2.json")
data = json.load(f)
f.close()

# Read video with OpenCV.
cap=cv2.VideoCapture('../UPDRS_video/10_19/AlphaPose_1019case2姿勢平穩度.mp4')

## Get video info
RES=(round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fps = round(cap.get(cv2.CAP_PROP_FPS))

#create video writer to write result
output_video = "../UPDRS_result/alphapose/result.mp4"
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_video, fourcc, fps, RES)

#store each frame in video
frame_count = 0
img_list = []
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        frame_count+=1
        img_list.append(frame)

    else:
        break
cap.release()


In [ ]:
#used for cmu pose format (i.e., 18keypoints)

neck_hip_angle = []
patient_side = 1 #index 1 for left person, and index 2 for right person
hip = None

if patient_side == 1: #if the patient is on the left side, use Lhip to calculate the angle
    hip = "LHip"
else: #if the patient is on the right side, use Rhip to calculate the angle
    hip = "RHip"


left_person_elbow_wrist_angle = []
right_person_elbow_wrist_angle = []
left_person_neck_Rhip_angle = []
right_person_neck_Rhip_angle = []
left_person_neck_Lhip_angle = []
right_person_neck_Lhip_angle = []
vertical_vector = [0,-10]
vertical_vector_down = [0,10]
for i in data:
    for j in data[i]["bodies"]:
        neck_x = j["joints"][keypoint_map_18["Neck"]]
        neck_y = j["joints"][keypoint_map_18["Neck"]+1]
        Rhip_x = j["joints"][keypoint_map_18["RHip"]]
        Rhip_y = j["joints"][keypoint_map_18["RHip"]+1]
        Lhip_x = j["joints"][keypoint_map_18["LHip"]]
        Lhip_y = j["joints"][keypoint_map_18["LHip"]+1]
        Rwrist_x = j["joints"][keypoint_map_18["RWrist"]]
        Rwrist_y = j["joints"][keypoint_map_18["RWrist"]+1]
        Relbow_x = j["joints"][keypoint_map_18["RElbow"]]
        Relbow_y = j["joints"][keypoint_map_18["RElbow"]+1]
        
        Rhip_neck_vector = [neck_x-Rhip_x, neck_y-Rhip_y]
        Lhip_neck_vector = [neck_x-Lhip_x, neck_y-Lhip_y]
        elbow_wrist_vector = [Rwrist_x-Relbow_x, Rwrist_y-Relbow_y]

        if(j["joints"][-1] == 1):
            left_person_neck_Rhip_angle.append(angle(vertical_vector,Rhip_neck_vector))
            left_person_neck_Lhip_angle.append(angle(vertical_vector,Lhip_neck_vector))
            left_person_elbow_wrist_angle.append(angle(vertical_vector_down,elbow_wrist_vector))

        elif(j["joints"][-1] == 2):
            right_person_neck_Rhip_angle.append(angle(vertical_vector,Rhip_neck_vector))
            right_person_neck_Lhip_angle.append(angle(vertical_vector,Lhip_neck_vector))
            right_person_elbow_wrist_angle.append(angle(vertical_vector_down,elbow_wrist_vector))
            
#decide the parent side base on elbow_wrist_vector
if max(right_person_elbow_wrist_angle[:100]) > max(left_person_elbow_wrist_angle[:100]):
    neck_hip_angle = left_person_neck_Lhip_angle
else:
    neck_hip_angle = right_person_neck_Rhip_angle

plt.title("neck_hip_angle")
plt.xlabel("frame")
plt.ylabel("angle")
plt.plot(neck_hip_angle)


In [ ]:
plt.title("right person")
plt.xlabel("frame")
plt.ylabel("angle")
plt.plot(right_person_elbow_wrist_angle)

In [ ]:
#denoise the data before find peaks
after_filt = savgol_filter(neck_hip_angle,31,5)

middle = (min(neck_hip_angle) + max(neck_hip_angle))/2
prominence = np.quantile(neck_hip_angle,0.75) - np.quantile(neck_hip_angle,0.25)

peaks, _ = signal.find_peaks(after_filt,height=middle,prominence=prominence)

plt.title("neck_hip_angle")
plt.xlabel("frame")
plt.ylabel("angle")

plt.plot(after_filt)
plt.plot(peaks,np.array(after_filt)[peaks],"x")

In [ ]:
#write result to video
action_count = 0
max_angle = 0
for i in range(len(img_list)):
    if( len(peaks) > action_count and i == peaks[action_count]):
        max_angle = after_filt[i]
        action_count += 1
    cv2.putText(img_list[i],f'neck_hip_angle:{after_filt[i]:.2f}',(10, 70), cv2.FONT_HERSHEY_SIMPLEX,1.5, (255, 0, 0), 3, cv2.LINE_AA)
    cv2.putText(img_list[i],f'max_angle:{max_angle:.2f}',(10, 120), cv2.FONT_HERSHEY_SIMPLEX,1.5, (255, 0, 0), 3, cv2.LINE_AA)
    out.write(img_list[i])

out.release()